In [0]:
# Databricks notebook: 05_gold_bi_views
# Default language: Python

catalog = "cscie103_catalog"
schema  = "final_project"

spark.sql(f"USE {catalog}.{schema}")

print("Using", catalog + "." + schema)

Using cscie103_catalog.final_project


In [0]:
%sql
CREATE OR REPLACE VIEW vw_monthly_sales_oil AS
WITH daily AS (
  SELECT
    date,
    SUM(sales)      AS daily_sales,
    AVG(dcoilwtico) AS daily_oil_price
  FROM gold_daily_store_family
  GROUP BY date
),
monthly AS (
  SELECT
    date_trunc('month', date) AS month_start,
    SUM(daily_sales)          AS monthly_sales,
    AVG(daily_oil_price)      AS avg_monthly_oil_price
  FROM daily
  GROUP BY date_trunc('month', date)
)
SELECT
  month_start,
  monthly_sales,
  avg_monthly_oil_price
FROM monthly
ORDER BY month_start;

In [0]:
 
%sql
CREATE OR REPLACE VIEW vw_yoy_family_2017 AS
WITH yearly_family AS (
  SELECT
    family,
    YEAR(date) AS year,
    SUM(sales) AS yearly_sales
  FROM gold_daily_store_family
  WHERE date >= '2016-01-01' AND date < '2018-01-01'
  GROUP BY family, YEAR(date)
),
pivoted AS (
  SELECT
    family,
    MAX(CASE WHEN year = 2016 THEN yearly_sales END) AS sales_2016,
    MAX(CASE WHEN year = 2017 THEN yearly_sales END) AS sales_2017
  FROM yearly_family
  GROUP BY family
)
SELECT
  family,
  sales_2016,
  sales_2017,
  (sales_2017 - sales_2016) / sales_2016 * 100.0 AS yoy_growth_pct
FROM pivoted
WHERE sales_2016 IS NOT NULL;

Top Families vs Other; takes only those that are in the top 

In [0]:
%sql
CREATE OR REPLACE VIEW vw_daily_with_family_group AS
WITH family_sales AS (
  SELECT
    family,
    SUM(sales) AS total_sales
  FROM gold_daily_store_family
  GROUP BY family
),
top_families AS (
  SELECT family
  FROM family_sales
  ORDER BY total_sales DESC
  LIMIT 10
)
SELECT
  date,
  store_nbr,
  CASE
    WHEN family IN (SELECT family FROM top_families)
      THEN family
    ELSE 'Other'
  END AS family_group,
  sales,
  onpromotion,
  transactions,
  dcoilwtico,
  city,
  state,
  store_type,
  cluster,
  is_holiday
FROM gold_daily_store_family;

In [0]:
%sql
SELECT * FROM vw_monthly_sales_oil LIMIT 10;



month_start,monthly_sales,avg_monthly_oil_price
2013-01-01T00:00:00.000Z,1.0327624736909498E7,94.75666666666616
2013-02-01T00:00:00.000Z,9658959.777436802,95.3089473684208
2013-03-01T00:00:00.000Z,1.1428497037487496E7,92.9385000000007
2013-04-01T00:00:00.000Z,1.09934647380118E7,92.02136363636345
2013-05-01T00:00:00.000Z,1.1985564512041297E7,94.50954545454515
2013-06-01T00:00:00.000Z,1.22995831223716E7,95.77250000000033
2013-07-01T00:00:00.000Z,1.1631952525629796E7,104.67090909090979
2013-08-01T00:00:00.000Z,1.17377889194727E7,106.57272727272726
2013-09-01T00:00:00.000Z,1.1792933231867898E7,106.28949999999948
2013-10-01T00:00:00.000Z,1.1775620360519098E7,100.53826086956431


In [0]:
%sql
SELECT * FROM vw_yoy_family_2017 LIMIT 10;


family,sales_2016,sales_2017,yoy_growth_pct
AUTOMOTIVE,140137.0,93088.0,-33.57357443073564
BABY CARE,5354.0,2776.0,-48.1509152035861
BEAUTY,103238.0,68372.0,-33.772448129564694
BEVERAGES,6.4018448E7,4.4858817E7,-29.928296606003318
BOOKS,4203.0,2337.0,-44.39685938615275
BREAD/BAKERY,1.061816127786495E7,6981076.827319983,-34.25343009365454
CELEBRATION,288954.0,168606.0,-41.64953591229054
CLEANING,2.342604E7,1.5791284E7,-32.590894577145775
DAIRY,1.758158E7,1.1692401E7,-33.496301242550444
DELI,5934369.07009801,3964842.1068360033,-33.18848120158254


In [0]:
%sql
SELECT * FROM vw_daily_with_family_group LIMIT 10;

date,store_nbr,family_group,sales,onpromotion,transactions,dcoilwtico,city,state,store_type,cluster,is_holiday
2013-01-01,1,Other,0.0,0,null,null,Quito,Pichincha,D,13,1
2013-01-01,1,Other,0.0,0,null,null,Quito,Pichincha,D,13,1
2013-01-01,1,Other,0.0,0,null,null,Quito,Pichincha,D,13,1
2013-01-01,1,BEVERAGES,0.0,0,null,null,Quito,Pichincha,D,13,1
2013-01-01,1,Other,0.0,0,null,null,Quito,Pichincha,D,13,1
2013-01-01,1,BREAD/BAKERY,0.0,0,null,null,Quito,Pichincha,D,13,1
2013-01-01,1,Other,0.0,0,null,null,Quito,Pichincha,D,13,1
2013-01-01,1,CLEANING,0.0,0,null,null,Quito,Pichincha,D,13,1
2013-01-01,1,DAIRY,0.0,0,null,null,Quito,Pichincha,D,13,1
2013-01-01,1,DELI,0.0,0,null,null,Quito,Pichincha,D,13,1
